## Training Transformer-based model on Arabic poetry
By Mohammed Alneyadi (December 2023)

### 0. Import packages

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(123)

### 1. Load & prepare our data

In [2]:
input_file_path = 'data/preprocessed_data.txt'
with open(input_file_path, 'r') as f:
    text = f.read()

print(f"length of dataset in characters: {len(text):,}")
print(f"length of unique characters: {len(set(text))}")

length of dataset in characters: 34,389,599
length of unique characters: 38


In [3]:
# unique characters that occur in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(' '.join(chars))


   ء آ أ ؤ إ ئ ا ب ة ت ث ج ح خ د ذ ر ز س ش ص ض ط ظ ع غ ف ق ك ل م ن ه و ى ي


In [4]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [5]:
# create train and test splits (90-10 split)
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

### 2. Define hyperparameters

In [6]:
parameters_sets = {
  'set1': {
      'n_layer':4,
      'n_head': 4,
      'batch_size':32,
  },
  'set2': {
      'n_layer':6,
      'n_head': 6,
      'batch_size':64,
  },
  'set3': {
      'n_layer':8,
      'n_head': 8,
      'batch_size':128,
  }}

In [7]:
# fixed hyperparameters
block_size = 256
max_iters = 5000
eval_interval = 250
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
dropout = 0.2

### 3. Design our transformer model

In [8]:
def get_batch(split, batch_size):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss(batch_size):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
# self-attention layer
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out

In [11]:
# multi-head attention layer
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [12]:
# feed-forward layer
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [13]:
# transformer block
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [14]:
# simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self, n_head, n_layer):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

### 4. Train our models

#### A. Using parameter set 1

In [ ]:
model = BigramLanguageModel(n_head = parameters_sets['set1']['n_head'],
                            n_layer = parameters_sets['set1']['n_layer'])
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


7.221542 M parameters


In [ ]:
# train model
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(batch_size = parameters_sets['set1']['batch_size'])
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train', batch_size = parameters_sets['set1']['batch_size'])

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 3.8395, val loss 3.8380
step 250: train loss 2.6483, val loss 2.6426
step 500: train loss 2.3094, val loss 2.3037
step 750: train loss 2.1512, val loss 2.1454
step 1000: train loss 2.0356, val loss 2.0270
step 1250: train loss 1.9739, val loss 1.9683
step 1500: train loss 1.9286, val loss 1.9265
step 1750: train loss 1.8939, val loss 1.8895
step 2000: train loss 1.8723, val loss 1.8675
step 2250: train loss 1.8487, val loss 1.8399
step 2500: train loss 1.8346, val loss 1.8303
step 2750: train loss 1.8250, val loss 1.8181
step 3000: train loss 1.8071, val loss 1.8013
step 3250: train loss 1.8005, val loss 1.7957
step 3500: train loss 1.7881, val loss 1.7832
step 3750: train loss 1.7811, val loss 1.7809
step 4000: train loss 1.7708, val loss 1.7702
step 4250: train loss 1.7679, val loss 1.7690
step 4500: train loss 1.7579, val loss 1.7551
step 4750: train loss 1.7534, val loss 1.7514
step 4999: train loss 1.7452, val loss 1.7463


In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


فرح نفوس أدريسهم بسيدهم    زينت كما قوم قد طلبت علمي
لم أفني بالدهر علم فرطه    أنرخ علم خلقا لا تنمكا ظلمي
من سباب بار شمس الكمال جازهرهم    إذا هو والتجزى من حلمهم
فذى امر منها في فرط شماله    تولى اليك يناها لشرع عوانهم
بمعانق تخفى بها وكادت من الصبا    وكل بغدتي عن سما أسمى عجمتي
وكلا عمرتي لقلبي فلذلتي    شخفك تحت لا عيشي تكادمتي
وقلبي لغيابي اخفروا مني أهلي يا    نيدم بديعي داعي بالتحرمن النصي
فسرى وهيهة تضربا من    وجوه ناعي عن تمكيني بمينهم
قد بعدت من وصابات الله لنوى    
قمرات تلك الوصال بعد بعضنهم    كما لا تردو بكمال الجميل جواري
عمان على مقالي دعات أقوى أساطعي    يسيد به الصواب هذي فأيامي قالم

أليس قرنا يافنا قل سما عيدي أضرما    إلى لك فللصب في ظه
واحمل أمره ما هو زينب جوبي صباح روعة    
لا ترق يضيء البقاء لذ تضرئب ندو العلى    وليس نهاضا أو تمر نختدر براح لفرس
أقراني بار ألتقيه كم فخز عزه    والله قول توله والفن يفطنه القطع موزج
أنى والأرض بالبيضاء والمفخر لنصرا    أكرم بقرتني بكأ من قر وظلز
أيهمن بحاه وما بين كل كمالي    فيلحق به وأمحى تحصيح نقالي
أمن قبلكم قد طاعت من 

#### B. Using parameter set 2

In [ ]:
model = BigramLanguageModel(n_head = parameters_sets['set2']['n_head'],
                            n_layer = parameters_sets['set2']['n_layer'])
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


10.768166 M parameters


In [ ]:
# train model
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(batch_size = parameters_sets['set2']['batch_size'])
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train', batch_size = parameters_sets['set2']['batch_size'])

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 3.8332, val loss 3.8349
step 250: train loss 2.6547, val loss 2.6496
step 500: train loss 2.2920, val loss 2.2885
step 750: train loss 2.0793, val loss 2.0743
step 1000: train loss 1.9658, val loss 1.9607
step 1250: train loss 1.8920, val loss 1.8883
step 1500: train loss 1.8421, val loss 1.8388
step 1750: train loss 1.8118, val loss 1.8088
step 2000: train loss 1.7855, val loss 1.7829
step 2250: train loss 1.7662, val loss 1.7653
step 2500: train loss 1.7518, val loss 1.7476
step 2750: train loss 1.7338, val loss 1.7344
step 3000: train loss 1.7224, val loss 1.7234
step 3250: train loss 1.7119, val loss 1.7158
step 3500: train loss 1.6985, val loss 1.7011
step 3750: train loss 1.6898, val loss 1.6924
step 4000: train loss 1.6817, val loss 1.6853
step 4250: train loss 1.6727, val loss 1.6739
step 4500: train loss 1.6619, val loss 1.6647
step 4750: train loss 1.6513, val loss 1.6555
step 4999: train loss 1.6447, val loss 1.6501


In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


كلام اسم لشكوى ووبلة العلا    تتقته نوارا وبأد قبا
وأشكال كأسكم في خضاب    وأطوع قلب منكم سبا
تقول الأمم بالخكلة والتو    وباللحطة ختم وذهبا
ولدها فضحى غوايا ختمان    من والدنيا بجميع أجمعها
وقد كنت بنادية بين النبي    ووليت بالاقتضان قبل شهبا
قرأت محتدا من فرش الشهب    وحلما حاسرا في العزب عضبا
وان قبل من جنود سليم    وارتع الى والكنز قد خرقا
وابتبعت الرى قبلي فيهم    روحي فكل الجليت بأبلحا
لو انجم بين زهر فيهم    من يومه فراش الجمع مرهبا
إذ تقيهم بالنسج الذكريا    ما اغتنم يحمل باتفا
كل يوم امحص عنه فغير لي    باحات ياما وبان الحروبا
وشدل لأنملق وجهه    صوبا وهو ذو الإنسان تضرك
وقد نزعنا ذي يومه    وو ارتياح لقلبي منك النوبا
حطام النفس فكرول لتجاريا    وليس لولا الغوار أمول مغرب
ذو الرأى ما قله مأنوب شعب    وعند لم يبدع ملوكا مولمونه
ابنفوا بالمكارم من ليس مكارم    والكريم الجسم مؤروما وحسبك مركب
إن أهلي لائم ورداء فلم تتك    لبدر في الأفلاك الأنابة والصبا
كودي ذو البلبس الكريم اليوم حلوه    يقبل الستر بلباب اللبس متازب
سبحانها بكر اليقين قليل    فلا يغفل فودفعها مجرون قلبي
دوحي بنا

#### C. Using parameter set 3

In [15]:
model = BigramLanguageModel(n_head = parameters_sets['set3']['n_head'],
                            n_layer = parameters_sets['set3']['n_layer'])
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


14.31479 M parameters


In [16]:
# train model
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(batch_size = parameters_sets['set3']['batch_size'])
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train', batch_size = parameters_sets['set3']['batch_size'])

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 3.7908, val loss 3.7897
step 250: train loss 2.6599, val loss 2.6541
step 500: train loss 2.2342, val loss 2.2295
step 750: train loss 1.9939, val loss 1.9875
step 1000: train loss 1.8869, val loss 1.8857
step 1250: train loss 1.8228, val loss 1.8183
step 1500: train loss 1.7812, val loss 1.7815
step 1750: train loss 1.7520, val loss 1.7515
step 2000: train loss 1.7343, val loss 1.7353
step 2250: train loss 1.7081, val loss 1.7097
step 2500: train loss 1.6925, val loss 1.6958
step 2750: train loss 1.6750, val loss 1.6788
step 3000: train loss 1.6586, val loss 1.6615
step 3250: train loss 1.6453, val loss 1.6502
step 3500: train loss 1.6304, val loss 1.6379
step 3750: train loss 1.6150, val loss 1.6214
step 4000: train loss 1.5984, val loss 1.6085
step 4250: train loss 1.5850, val loss 1.5957
step 4500: train loss 1.5710, val loss 1.5834
step 4750: train loss 1.5586, val loss 1.5702
step 4999: train loss 1.5477, val loss 1.5616


In [17]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


تعلم أن تشكو لشأوك قولهم حتى    تجب إلى النار اني بالمجامين والنين
إلى أنه ما قال لي غصن عتيقة    وعن غايتي يشكون في الخل والتين
يطل على الهاتي والوغى حادثا وهوى    يلوح بيثوب الذكر بالدموع تمامين
ولي بعد محمود لك الصيد جازعا    خطابين بين العالمين الدين
منار يريناه بالجو مجنبه    والمجتري ببدا هو العيون
ودام على أيى ابن الحمام ففي    كل الضمير أبا الإعدام تميين
وأنت أدري ثورة تبدي لذرة    لولا ارفعت ثم تكبين نكيين
أم قد سلبت عن كبير الخلق فانثني    وكنت أكبر للرشاد توكيني
وقد بدا لمخلوق وسبر تدنى علا    سبق الحسين إذا ما بدا الحدثين
ومن عجيب دنا خيلا على أين    وأصول قبل أودى بالمتين
ومن حقيقة لم يهمره الله لدى    مصر بل من ضرام الجن والبهن
ومن بأبيك آل الكذبيات سامي    والجوم يحتمل بالحزن الرنين
فلقد جزات والسهو يمين على    قدس الأساوين بالدع والنحن
ومن يكون الموت بالخمرة التي    لم يحسن الآيات يشهى للسابغة
واحذر بنصر الأرض بالحر البلا    أني أقول الأرض فبالإنسان
ومها للفضل أهلى بالتنبي وقا    وسؤال بالتمويه وأضعاني
ورب المبادي والفضل الذي به    شاء الغناءين فيه أيمان
قد يختالبون با